In [3]:
import nltk
import re
import multiprocessing as mp
import plotly.express as px

In [20]:
def writeToFile(filename, left_tokenize, right_tokenize, query):
    obj=[query],left_tokenize,right_tokenize

    with open(filename, 'a', encoding='utf8') as fin:
        fin.write(str(obj) + "\n")

In [18]:
# Creates a new file, takes the base concordance file and runs POS tagging.
# This code strips the concordance down to left and right of the query (word concordance query)
def stripConcordance(query):
    stripped_string = "".lower()
    with open('../textFiles/Pronoun/concordance_'+ query + '.txt', 'r', encoding='utf8') as f:
        for line in f.readlines():
            
            # Strips the left and right side of the concordance of unwanted parts.
            left = line[line.find("left_print="):line.find("right_")].strip("left_print=")
            left_re = re.sub('[^A-Za-z0-9 ]+', '', left)
            right = line[line.find("right_print="):line.find("line=")].strip("right_print=")
            right_re = re.sub('[^A-Za-z0-9 ]+', '', right)
            
            # Combines the left and right side of the concordance into a single string.
            stripped_string += left_re
            stripped_string += " "
            stripped_string += right_re
            
            #left_tokenize = nltk.pos_tag(nltk.word_tokenize(left_re))
            #right_tokenize = nltk.pos_tag(nltk.word_tokenize(right_re))
            #writeToFile('../textFiles/Pronoun/POS-concordance-'+ query +'.txt', left_tokenize, right_tokenize, query)
            writeToFile('../textFiles/Pronoun/POS-concordance-'+ query +'.txt', left_re, right_re, query)
        
        #return posTag(stripped_string, query)

In [7]:
# Removes words used to obtain data as well as any words that have snuck into the concordance.
def posStripper(stripped_string):
    stripped_string = stripped_string.replace("conspiracy", "")
    stripped_string = stripped_string.replace("propaganda", "")
    stripped_string = stripped_string.replace("Trump", "")
    stripped_string = stripped_string.replace("fake", "")
    stripped_string = stripped_string.replace("realDonald", "")
    stripped_string = stripped_string.replace("misinformation", "")
    stripped_string = stripped_string.replace("https", "")
    return stripped_string

In [207]:
def posTag(stripped_string, query):
    # calls function to remove certain words used to obtain data.
    stripped_string = posStripper(stripped_string)
    
    # POS tag the entire concordance.
    token = nltk.pos_tag(nltk.word_tokenize(stripped_string))

    nouns = getNouns(token, query)
    verbs = getVerbs(token, query)
    adj = get_adjective(token, query)
    all = get_all(token, query)
    return all

In [200]:
def get_all(token, query):
    most_common_any = []
    for i, j in token:
        if len(i) > 4:
            most_common_any.append(i)

    any_frq = nltk.FreqDist(any for any in most_common_any)
    most_common_any = any_frq.most_common(20)
    return most_common_any

In [8]:
# get all nouns and return the 20 most common
def getNouns(token, query):
    most_common_nouns = []
    for i, j in token:
        if 'NN' in j and len(i) > 4:
            most_common_nouns.append(i)
    noun_frq = nltk.FreqDist(noun for noun in most_common_nouns)
    most_common_nouns = noun_frq.most_common(20)
    return most_common_nouns

In [52]:
# get all verbs and return the 20 most common
def getVerbs(token, query):
    most_common_verbs = []

    for i, j in token:
        if 'V' in j and len(i) >= 4:
            most_common_verbs.append(i)

    verb_frq = nltk.FreqDist(verb for verb in most_common_verbs)
    most_common_verbs = verb_frq.most_common(20)

    return most_common_verbs

In [81]:
def get_adjective(token, query):
    most_common_adj = []
    for i, j in token:
        if 'J' in j and len(i) >= 3:
            most_common_adj.append(i)
    adj_frq = nltk.FreqDist(verb for verb in most_common_adj)
    most_common_adj = adj_frq.most_common(20)

    return most_common_adj

In [1]:
# Function to sort the list of tuples by its second item
def Sort_Tuple(tup): 
    # getting length of list of tuples
    lst = len(tup)
    for i in range(0, lst):
        for j in range(0, lst-i-1):
            if (tup[j][1] > tup[j + 1][1]): 
                temp = tup[j] 
                tup[j]= tup[j + 1] 
                tup[j + 1]= temp 
    return tup

# Function to remove single letter 'e' which cropped up in Russian dataset.
def remove_single_letter_words(tup):
    for i in tup:
        if i[0] == 'е':
            tup.remove(i)
    return tup

In [22]:
# Saves concordance for each pronoun to a variable.
# Will automate this and make it cleaner at some point

vocab = ["I", "They", "We", "My", "Us", "You", "Me"]

verbs_I = stripConcordance(vocab[0])[0]
verbs_They = stripConcordance(vocab[1])[0]
verbs_We = stripConcordance(vocab[2])[0]
verbs_My = stripConcordance(vocab[3])[0]
verbs_Us = stripConcordance(vocab[4])[0]
verbs_You = stripConcordance(vocab[5])[0]
verbs_Me = stripConcordance(vocab[6])[0]

TypeError: 'NoneType' object is not subscriptable

In [55]:
print(type(verbs_Me))

<class 'list'>


In [62]:
# Sorts the tuple list in order of verb frequency

verbs_I = Sort_Tuple(verbs_I)
verbs_They = Sort_Tuple(verbs_They)
verbs_We = Sort_Tuple(verbs_We)
verbs_My = Sort_Tuple(verbs_My)
verbs_Us = Sort_Tuple(verbs_Us)
verbs_You = Sort_Tuple(verbs_You)
verbs_Me = Sort_Tuple(verbs_Me)

In [63]:
import plotly.express as px
import math

# Constructors
verbs = []
verbCounter=[]

# Simple function to find a percentage.
def findPercentage(num, total):
    quotient = num / total
    return quotient*100

# returns total number of verbs associated with a pronoun.
def findTotalFreq(verbListTuple):
    total=[]
    for i,j in verbListTuple:
        total.append(j)
    return sum(total)

# Function to find the rounded down percent of a single verb in
# comparison to all other verbs associated with the pronoun,
def findPerecentTotalFreq(verbListTuple):
    verbCounterPercentage=[]
    total = findTotalFreq(verbListTuple)
    temp = []
    for i,j in verbListTuple:
        temp.append(math.floor(findPercentage(j, total)))
    verbCounterPercentage.append(temp)
    
    return verbCounterPercentage

# Function to split a list of tuples into two seperate ordered lists.
# Here we add the verb to "verbs" and the frequency of the verb to "verbCounter"
def splitTuple(verbListTuples):
    temp = []
    tempCount = []

    for i, j in enumerate(verbListTuples):
        temp.append(j[0])
        tempCount.append(j[1])
        if i == 19:
            verbCounter.append(tempCount)
            verbs.append(temp)
            temp=[]
            tempCount=[]
            break

# Uses the ST function to seperate the verb and frequency count and add them to seperate lists.
# splitTuple(verbs_I)
splitTuple(verbs_Me)
# splitTuple(verbs_You)

# Using the FPTF function to add each perecnt frequency to an ordered list.
# findPerecentTotalFreq(verbs_I)
findPerecentTotalFreq(verbs_Me)
# findPerecentTotalFreq(verbs_You)

In [222]:
import plotly.express as px
import pandas as pd

# Function which creates a dataframe containing
# the verb tuple and associated pronoun. 
def dataframer(words_tuple, pnoun):
    data = {}
    counter = []
    words = []

    for i in words_tuple:
        j, k = i
        words.append(j)
        counter.append(k)

    for pronoun in range(len(words)):
        data["Pronoun"] = pnoun
        data["Word"] = words
        data["Count"] = counter

    df_new = pd.DataFrame(data=data)
    return df_new

Below function creates a graph based on the tuple of data provided.
The end goal is to produce a stacked bar chart displaying the frequency
of commonly occuring words associated with the concorded word.
For example 

- [a list containing a tuple]
- (a tuple containing a list of words and their frequency, and the number of results the user wants to see)

** Need to return the result as a percentage not a count **


In [218]:
# Function takes a tuple of the (list of words) and the (concorded word) and the most frequent (X)
# That the user wants returned.
# Returns a stacked bar chart of the processed data.
def bar_stacked(list_of_tuples, x, title):
    x = -abs(x)
    df = pd.DataFrame(columns=["Base Word", "Word", "Count"])
    # i_df = dataframer(verbs_I[-5:], "I")
    new = df

    for i, j in list_of_tuples:
        temp = dataframer(i[x:], j)
        new = new.append(temp)

    new = new.sort_values(by=['Count'], ascending=True)

    fig = px.bar(new, x="Base Word", y="Count", color="Pronoun", title=title, text=new['Word'], color_discrete_sequence=px.colors.sequential.Viridis)
    fig.update_layout(hovermode="closest")

    fig.show()

In [227]:
verb_list_and_pronoun = [(verbs_I, "I"), (verbs_They, "They"), (verbs_We, "We"), (verbs_My, "My"), (verbs_Us, "Us"), (verbs_You, "You"), (verbs_Me, "Me")]
bar_stacked(verb_list_and_pronoun, 10, "10 Most Common Verbs Found Alongside English Pronouns")

In [197]:
# Function to automate concordance stripping
# Takes a list of words which are the relevant files the concordance was carried out on.
def automatic_concordance_stripping(vocab):
    ls = []
    for i in vocab:
        ls.append((Sort_Tuple(stripConcordance(i)), i))
    return ls

In [211]:
vocab = ["have", "believe", "think", "know", "tell", "need", "want"]
ls_vrb_adj = automatic_concordance_stripping(vocab)

In [230]:
ls_of_vars = []

for i in ls_vrb_adj:
    ls_of_vars.append(i)
    
bar_stacked(ls_of_vars, 10, "10 Most Common Words (all types) Alongside The Most Frequent Verbs From Concorded Pronouns.")